In [8]:
from Crypto.Cipher import AES
from Crypto import Random

KEY_LENGTH = 16  # AES128
BLOCK_SIZE = AES.block_size

_random_gen = Random.new()
_key = _random_gen.read(KEY_LENGTH)


def _add_padding(msg):
	pad_len = BLOCK_SIZE - (len(msg) % BLOCK_SIZE)
	padding = bytes([pad_len]) * pad_len
	return msg + padding


def _remove_padding(data):
	pad_len = data[-1]
	
	if pad_len < 1 or pad_len > BLOCK_SIZE:
		return None
	for i in range(1, pad_len):
		if data[-i-1] != pad_len:
			return None
	return data[:-pad_len]


def encrypt(msg):
	iv = _random_gen.read(AES.block_size)
	cipher = AES.new(_key, AES.MODE_CBC, iv)
	return iv + cipher.encrypt(_add_padding(msg))


def _decrypt(data):
	iv = data[:BLOCK_SIZE]
	cipher = AES.new(_key, AES.MODE_CBC, iv)
	return _remove_padding(cipher.decrypt(data[BLOCK_SIZE:]))


def is_padding_ok(data):
	return _decrypt(data) is not None


if __name__ == '__main__':
	#print("decrypted message:", _decrypt( ciphertext ) )
	print("USE attack.py!!")

USE attack.py!!


### sniff the cipher text with 64 byte

In [14]:
messages = (b'asdfghjkl;qwertyuiop[]zxcvbnm,./')
ciphertext = encrypt(messages)
print(len(ciphertext))
ciphertext

64


b"\xb2\xd5b2\x82Ht\xe7/x\x0c\xb3\xff\x9d\xbfgu\xfd$\xf2-G\xad\x08\xc5\x0c\xc8\xd6\xce4\xc7\xb0\nk\xba\xde\xa1\x83'\x17\x9f_f\x93\x97\x05\xc9\x89\xe1P\xd3\x9b\x80Q\x1fc\xdd\xd4\xa2.t\x97&\xcc"

### divide it into 4 cipher blocks

In [10]:
split_string = lambda x, n: [x[i:i+n] for i in range(0, len(x), n)]
blocks = split_string( ciphertext, BLOCK_SIZE )
blocks


[b'\xb8\x86rf4\x9b\xd0\\fl\x82\xb3\xa9\xe8\x8a{',
 b'\xa20\xd5\xf4(\xc1\x86\nD&vdZ\xdc~\xd8',
 b"'\xd9y\xa7a\xea\xe2\x8cg\x88\xd6\xb9+\xaa*<",
 b'\x9f$\x11\xc2BBk\xb6z\xd8\xc8o\x93\xc9\x8eJ']

### get the first n blocks only to deal with 
* for block_n = 1 we get `block 0 + block 1`

In [11]:
block_n = 1
spliced_ciphertext = blocks[block_n - 1] + blocks[block_n]
print(len(spliced_ciphertext))
spliced_ciphertext


32


b'\xb8\x86rf4\x9b\xd0\\fl\x82\xb3\xa9\xe8\x8a{\xa20\xd5\xf4(\xc1\x86\nD&vdZ\xdc~\xd8'

In [12]:
byte = 15 
new_pad_len = 16 - byte # we will pad with one byte

for padder_index in range( 1, new_pad_len ):
    hacked_ciphertext_tail += bytearray.fromhex('{:02x}'.format( new_pad_len ^ decoded_bytes[byte + padder_index]))

In [13]:

for i in range(0 , 255):
    attacker_str = i ^ spliced_ciphertext[byte]
    hacked_ciphertext = spliced_ciphertext[:byte] + attacker_str  +  hacked_ciphertext_tail + spliced_ciphertext[byte + 1 + new_pad_len - 1:]

TypeError: can't concat int to bytes

In [ ]:
# 1 - go throught each cipher block we have
# 2 - go throught each byte in this block

# 3 - go through 0 to 255 to bruteforce that block
#       a - construct the attack string
#       b - ask the oracle
#           - if true append to cracked cipher text
#           - if false do nothing

In [15]:
def attack(ciphertext):
    # define the gussed plain text
    gussed_plain_text = b''

    # define a split function to split the cipher text we have to block of 16 byte
    split_string = lambda x, n: [x[i:i+n] for i in range(0, len(x), n)]

    # split the cipher text we have to blocks of 16
    blocks = split_string(ciphertext , 16)
    for b in blocks:
        print(b)
    # go through each block (trying to decrypt it)
    for block_n in range(len(blocks)-1 , 0 , -1): # build pair of blocks starting from end of message
        print(block_n)
        pair_of_blocks = blocks[block_n - 1] + blocks[block_n]

        decoded_bytes = b's'*16
        for byte in range(16-1 , -1 , -1):
            # we will calculate the controlled padding of pad_len bytes 
            # if we trying to guess the last byte then we controlls 1 byte
            pad_len = 16 - byte
            
            # here we will store the calculated controlled fields
            hacked_cipher_text_tail = b''

            for padder_index in range(1 , pad_len):
                hacked_cipher_text_tail += (pad_len ^ decoded_bytes[byte + padder_index]).to_bytes(1 , 'big')
                print(hacked_cipher_text_tail)

            # brute force
            for i in range(0 , 256):
                attacker_str = i ^ spliced_ciphertext[byte]




attack(ciphertext)

b'\xb2\xd5b2\x82Ht\xe7/x\x0c\xb3\xff\x9d\xbfg'
b'u\xfd$\xf2-G\xad\x08\xc5\x0c\xc8\xd6\xce4\xc7\xb0'
b"\nk\xba\xde\xa1\x83'\x17\x9f_f\x93\x97\x05\xc9\x89"
b'\xe1P\xd3\x9b\x80Q\x1fc\xdd\xd4\xa2.t\x97&\xcc'
3
b'q'
b'p'
b'pp'
b'w'
b'ww'
b'www'
b'v'
b'vv'
b'vvv'
b'vvvv'
b'u'
b'uu'
b'uuu'
b'uuuu'
b'uuuuu'
b't'
b'tt'
b'ttt'
b'tttt'
b'ttttt'
b'tttttt'
b'{'
b'{{'
b'{{{'
b'{{{{'
b'{{{{{'
b'{{{{{{'
b'{{{{{{{'
b'z'
b'zz'
b'zzz'
b'zzzz'
b'zzzzz'
b'zzzzzz'
b'zzzzzzz'
b'zzzzzzzz'
b'y'
b'yy'
b'yyy'
b'yyyy'
b'yyyyy'
b'yyyyyy'
b'yyyyyyy'
b'yyyyyyyy'
b'yyyyyyyyy'
b'x'
b'xx'
b'xxx'
b'xxxx'
b'xxxxx'
b'xxxxxx'
b'xxxxxxx'
b'xxxxxxxx'
b'xxxxxxxxx'
b'xxxxxxxxxx'
b'\x7f'
b'\x7f\x7f'
b'\x7f\x7f\x7f'
b'\x7f\x7f\x7f\x7f'
b'\x7f\x7f\x7f\x7f\x7f'
b'\x7f\x7f\x7f\x7f\x7f\x7f'
b'\x7f\x7f\x7f\x7f\x7f\x7f\x7f'
b'\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f'
b'\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f'
b'\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f'
b'\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f'
b'~'
b'~~'
b'~~~'
b'~~~~'
b'~~~~~'
b'~~~~~~'
b